## Resources

* https://canvasapi.readthedocs.io
* https://canvas.instructure.com/doc/api/

# Importing from canvas-update

In [7]:
from api import get_course, split_folder_url, find_folder

In [26]:
config_file = '~/.config/canvasapi.conf'
url = 'https://uio.instructure.com/courses/4258/files/folder/quizzes'

API_URL, course_id, folder_name = split_folder_url(url)
course =  get_course(API_URL, course_id, config_file)

# find the folder
folder = find_folder(course, folder_name)
if not folder:
    sys.exit("Could not find folder '%s'. Full url: %s" %(folder_name, url))


In [27]:
for module in course.get_modules():
    print(module.name, module.id, module.items_url)

Uke 1 5248 https://uio.instructure.com/api/v1/courses/4258/modules/5248/items
Uke 2 5254 https://uio.instructure.com/api/v1/courses/4258/modules/5254/items


In [33]:
course.create_page(wiki_page = {"title":"Testing creating a page", \
                                "body":'<h1 id="dette-er-en-test">Dette er en test</h1><p>Versjon 1.</p>'}
                  )

Page(_requester=<canvasapi.requester.Requester object at 0x110fd60f0>, title=Testing creating a page, created_at=2019-09-13T18:14:09Z, created_at_date=2019-09-13 18:14:09+00:00, url=testing-creating-a-page, editing_roles=teachers, page_id=66357, last_edited_by={'id': 13657, 'display_name': 'Lex Nederbragt', 'avatar_image_url': 'https://uio.instructure.com/images/thumbnails/78465/9BZV9zaPiqdGS9vTMuzF7sAzpR3UM27Ku1XrpXLH', 'html_url': 'https://uio.instructure.com/courses/4258/users/13657'}, published=False, hide_from_students=True, front_page=False, html_url=https://uio.instructure.com/courses/4258/pages/testing-creating-a-page, todo_date=None, updated_at=2019-09-13T18:14:09Z, updated_at_date=2019-09-13 18:14:09+00:00, locked_for_user=False, body=<h1 id="dette-er-en-test">Dette er en test</h1><p>Versjon 1.</p>, course_id=4258)

In [62]:
url = 'https://canvas.instance.com/courses/12345/pages/page_name'
none, API_URL, course_id, rest, none = re.split(r'(.*)/courses/(\d*)/(.*)', url)
print(API_URL, course_id, rest)
#url = 'https://canvas.instance.com/courses/12345/files/folder/folder_name'
#none, API_URL, course_id, rest, none = re.split(r'(.*)/courses/(\d*)/(.*)', url)
#print(API_URL, course_id, rest)

https://canvas.instance.com 12345 pages/page_name


In [63]:
if rest.startswith('pages'):
    url_type = 'page'
    item_name = re.sub(r'^pages/', '', rest)
elif rest.startswith('files/folder'):
    url_type = 'folder'
    item_name = re.sub(r'^files/folder/', '', rest)
print(url_type, item_name)

page page_name


In [51]:
import api
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
%autoreload 2
url = 'https://canvas.instance.com/courses/12345/pages/page_name'
url = 'https://canvas.instance.com/courses/12345/files/folder/folder_name'
#url = 'https://canvas.instance.com/courses/12345/age/folder_name'
result = api.split_url(url)
print(result)

('https://canvas.instance.com', '12345', 'folder', 'folder_name')


------------

## Without importing from canvas-update

In [2]:
import os, sys
import configparser
# Import the Canvas class
from canvasapi import Canvas
from canvasapi import page
import re

In [3]:
url = 'https://uio.instructure.com/courses/5848/pages/testing-canvas-api'
none, API_URL, course_id, url_for_page, none = re.split('(.*)/courses/(.*)/pages/(.*)', url)
print(API_URL, course_id, url_for_page)

https://uio.instructure.com 5848 testing-canvas-api


In [4]:
canvas_instance = 'uio'
course_code = 'bios1100'
html_to_send = 'api_test.html'
title_for_page = 'Testing Canvas API'
# canvas uses the page title in lower case with dashes for spaces as url for the page
url_for_page = title_for_page.lower().replace(' ','-')

In [5]:
# load config file
config = configparser.ConfigParser()

config_file = '~/.config/canvasapi.conf'

result = config.read(os.path.expanduser(config_file))
if result == []:
    sys.exit("Error: could not open config file or config file was empty or malformed: " + config_file)

In [5]:
# Canvas API URL
try:
    API_URL = config['api_url'][canvas_instance]
except KeyError:
    print("Error: could not find the entry for Canvas instance '%s' in the 'api_url' section of the config file." %canvas_instance)

# Course ID
try:
    course_id = config[course_code]['course_id']
except KeyError:
    print("Error: could not find the 'api-key' entry in for course code '%s' in the 'courses' section of the config file." %course_code)

# Canvas API key
try:
    API_KEY = config[course_code]['api_key']
except KeyError:
    print("Error: could not find the 'api-key' entry for course code '%s' in the 'courses' section of the config file." %course_code)


Error: could not find the entry for Canvas instance 'uio' in the 'api_url' section of the config file.
Error: could not find the 'api-key' entry in for course code 'bios1100' in the 'courses' section of the config file.
Error: could not find the 'api-key' entry for course code 'bios1100' in the 'courses' section of the config file.


In [5]:
# Initialize a new Canvas object
canvas = Canvas(API_URL, API_KEY)

# get the course
course = canvas.get_course(API_URL, course_id, config_file)
course.name

'BIOS1100-1 18H Innføring i beregningsmodeller for biovitenskap'

In [30]:
# read new content
with open(html_to_send, 'r') as html_file:
    html_content = html_file.read()#.replace('\n', '')

# get the course page
try:
    test_page = course.get_page(url_for_page)
except:
    sys.exit("Error: could not find page '%s' on Canvas for updating." % url_for_page)

# update the course page
#api_call_result = test_page.edit(wiki_page = {"title":title_for_page, "body":html_content})

In [31]:
html_content.split("\n")[:-1]

['<h1 id="dette-er-en-test">Dette er en test</h1>', '<p>Versjon 12.</p>']

In [32]:
test_page.body.split("\n")[:-1]

['<h1 id="dette-er-en-test">Dette er en test</h1>', '<p>Versjon 12.</p>']

In [33]:
test_page.body.split("\n")[:-1] == html_content.split("\n")[:-1]

True

In [7]:
# testing whether the update has happened
# in which case the first part of the api_call_result is the updated html
html_content in api_call_result.body

True

In [8]:
sys.exit('Do not run below')

SystemExit: Do not run below

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
test_page = course.get_page('test2')

In [ ]:
course.create_page({"title":"api_test", "body":"Test vellykket"})

In [ ]:
test_page = course.get_page('api_test')

In [ ]:
test_page.body

In [ ]:
test_page.edit(wiki_page = {"body": "Tekst endret"})

In [ ]:
test_page = course.get_page('api_test')
test_page.body